In [1]:
# SQLite　通常ファイルにDBを作成する。どのOSでも動作するので、移植性が非常に効果的。":memory:"を指定するとメモリ上にDBを作成できる
# DBファイル名: enterprise.db, テーブル: zoo (動物園ビジネスを管理), テーブルの列名: critter(動物名) count(個体数) damages(損失)
import sqlite3
conn = sqlite3.connect('enterprise.db')
curs = conn.cursor()
curs.execute('''CREATE TABLE zoo (critter VARCHAR(20) PRIMARY KEY, count INT, damages FLOAT)''')

OperationalError: table zoo already exists

In [2]:
# 動物園に動物のデータを追加
curs.execute('INSERT INTO zoo VALUES("duck", 5, 0.0)')

In [3]:
curs.execute('INSERT INTO zoo VALUES("bear", 2, 1000.0)')

In [4]:
# プレースホルダーを使った安全な方法
ins = 'INSERT INTO zoo (critter, count, damages) VALUES(?, ?, ?)'
curs.execute(ins, ('weasel', 1, 2000.0))

In [5]:
# すべての動物の情報を引き出す
curs.execute('SELECT * FROM zoo')

In [6]:
rows = curs.fetchall()
print(rows)

[('duck', 5, 0.0), ('bear', 2, 1000.0), ('weasel', 1, 2000.0)]


In [7]:
# 個体順にソートして情報を取り出す
curs.execute('SELECT * FROM zoo ORDER BY count')

In [8]:
curs.fetchall()

[('weasel', 1, 2000.0), ('bear', 2, 1000.0), ('duck', 5, 0.0)]

In [9]:
# 降順でソートして取り出す
curs.execute('SELECT * FROM zoo ORDER BY count DESC')

In [10]:
curs.fetchall()

[('duck', 5, 0.0), ('bear', 2, 1000.0), ('weasel', 1, 2000.0)]

In [11]:
# 最も損失の大きい動物は何か?
curs.execute('''SELECT * FROM zoo WHERE damages = (SELECT MAX(damages) FROM zoo)''')

In [12]:
curs.fetchall()

[('weasel', 1, 2000.0)]

In [13]:
# SQLiteから離れる前にはクリーンアップ処理が必要(接続とカーソルを閉じる)
curs.close()
conn.close()

In [14]:
# MySQLで本格的なサーバーDBの運用
# PostgreSQLはMySQLよりも高度な操作が可能
# 様々な方言を持つSQLを跨ぐクロスSQLライブラリ->SQLAlchemy
import sqlalchemy as sa
# dialect + driver :// user : password@host : port / dbname
# ルート指定(/ or C:\\)をしなければ、相対パス
# dbnameを省略 or 'sqlite:///memory'でメモリ内にDBを作る

conn = sa.create_engine('sqlite://')

In [15]:
conn.execute('''CREATE TABLE zoo (critter VARCHAR(20) PRIMARY KEY, count INT, damages FLOAT)''')

In [16]:
ins = 'INSERT INTO zoo (critter, count, damages) VALUES(?, ?, ?)'
conn.execute(ins, 'duck', 10, 0.0)

In [17]:
conn.execute(ins, 'bear', 2, 1000.0)

In [18]:
conn.execute(ins, 'weasel', 1, 2000.0)

In [19]:
# 今書き込んだすべての情報を表示するようにデータベースに要求
rows = conn.execute('SELECT * FROM zoo')

In [20]:
print(rows)

In [21]:
for row in rows:
    print(row)

('duck', 10, 0.0)
('bear', 2, 1000.0)
('weasel', 1, 2000.0)


In [22]:
# SQL表現言語
import sqlalchemy as sa
conn = sa.create_engine('sqlite://')
meta = sa.MetaData()
zoo = sa.Table('zoo', meta,
              sa.Column('critter', sa.String, primary_key=True),
              sa.Column('count', sa.Integer),
              sa.Column('damages', sa.Float)
              )
meta.create_all(conn)

In [23]:
conn.execute(zoo.insert(('bear', 2, 1000.0)))

In [24]:
conn.execute(zoo.insert(('weasel', 1, 2000.0)))

In [25]:
conn.execute(zoo.insert(('duck', 10, 0)))

In [26]:
result = conn.execute(zoo.select())

In [27]:
rows = result.fetchall()
print(rows)

[('bear', 2, 1000.0), ('weasel', 1, 2000.0), ('duck', 10, 0.0)]


In [28]:
# ORM(オブジェクト関係マッピング)がSQL表現言語を使う
# python操作 - ORM - データベース
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base

In [29]:
# 接続を開設
conn = sa.create_engine('sqlite:///zoo.db')

In [30]:
# SQLAlchemyのORMを使って、データベースを利用
Base = declarative_base()
class Zoo(Base):
    __tablename__ = 'zoo'
    critter = sa.Column('critter', sa.String, primary_key=True)
    count = sa.Column('count', sa.Integer)
    damages = sa.Column('damages', sa.Float)
    
    def __init__(self, critter, count, damages):
        self.critter = critter
        self.count = count
        self.damages = damages
    
    def __repr__(self):
        return "<Zoo({}, {}, {})>".format(self.critter, self.count, self.damages)

In [31]:
# データベースとテーブルを作成
Base.metadata.create_all(conn)

In [32]:
first = Zoo('duck', 10, 0.0)
second = Zoo('bear', 2, 1000.0)
third = Zoo('weasel', 1, 2000.0)
first

<Zoo(duck, 10, 0.0)>

In [33]:
# データベースとやり取りするためのセッション
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=conn)
session = Session()

In [34]:
# オブジェクトを追加する
session.add(first)
session.add_all([second, third])

In [35]:
# 最後に強制的にすべての処理を完了させる
session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: zoo.critter [SQL: 'INSERT INTO zoo (critter, count, damages) VALUES (?, ?, ?)'] [parameters: (('duck', 10, 0.0), ('bear', 2, 1000.0), ('weasel', 1, 2000.0))] (Background on this error at: http://sqlalche.me/e/gkpj)

In [36]:
# NoSQLデータストア

# キーバリューストア
# ・キーに値を代入でき、代入された値は自動的にディスク上のデータベースにほぞんされる。
# ・キーから値を取得できる
import dbm

In [37]:
db = dbm.open('definitions', 'c')

In [38]:
db['mustard'] = 'yellow'
db['ketchap'] = 'red'
db['pesto'] = 'green'

In [39]:
len(db)

3

In [40]:
db['pesto']

b'green'

In [41]:
db.close()

In [42]:
db = dbm.open('definitions', 'r')

In [43]:
db['mustard']

b'yellow'

In [100]:
# Redis データ構造サーバー　リモート同士のPythonプログラムのデータ交換に役立つ
import redis
print(redis.__version__)
conn = redis.Redis('localhost')

2.10.6


In [101]:
# 文字列
conn.keys('*')

[b'carats', b'pie', b'fever', b'song', b'zoo', b'secret', b'cordial']

In [102]:
conn.set('secret', 'ni!')

True

In [103]:
conn.set('carats', 24)

True

In [104]:
conn.set('fever', '101.5')

True

In [105]:
conn.get('secret')

b'ni!'

In [106]:
conn.get('carats')

b'24'

In [107]:
conn.get('fever')

b'101.5'

In [108]:
# キーが存在しなときに限り値を設定する
conn.setnx('secret', 'icky-icky-icky-ptang-zoop-boing!')

False

In [109]:
conn.get('secret')

b'ni!'

In [110]:
# もともとの値を返すとともに、新しい値を設定する
conn.getset('secret', 'icky-icky-icky-ptang-zoop-boing!')

b'ni!'

In [111]:
conn.get('secret')

b'icky-icky-icky-ptang-zoop-boing!'

In [112]:
# 部分文字列を取り出す 先頭0, 末尾-1
conn.getrange('secret', -6, -1)

b'boing!'

In [113]:
# 部分文字列の置換
conn.setrange('secret', 0, 'ICKY')

32

In [114]:
conn.get('secret')

b'ICKY-icky-icky-ptang-zoop-boing!'

In [115]:
# 一度に複数のキーを設定する
conn.mset({'pie': 'cherry', 'cordial': 'sherry'})

True

In [116]:
# 一度に複数の値を取得する
conn.mget(['fever', 'carats'])

[b'101.5', b'24']

In [117]:
# キーの削除
conn.delete('fever')

1

In [118]:
# incr() or incrbyfloat() インクリメント
# decr() デクリメント
conn.incr('carats')

25

In [119]:
conn.incr('carats', 10)

35

In [120]:
conn.decr('carats')

34

In [121]:
conn.decr('carats', 15)

19

In [122]:
conn.set('fever', '101.5')

True

In [123]:
conn.incrbyfloat('fever')

102.5

In [124]:
conn.incrbyfloat('fever', 0.5)

103.0

In [125]:
# decrbyfloat()はない
conn.incrbyfloat('fever', -2.0)

101.0

In [126]:
# Redisリストは文字列しか格納できない
conn.lpush('zoo', 'bear')

5

In [127]:
conn.lpush('zoo', 'alligator', 'duck')

7

In [128]:
# 前か後ろに挿入する
conn.linsert('zoo', 'before', 'bear', 'beaver')

8

In [129]:
conn.linsert('zoo', 'after', 'bear', 'cassowary')

9

In [130]:
# 指定した位置に挿入する　※リストはすでに存在しえいなければならない
conn.lset('zoo', 2, 'marmoset')

True

In [131]:
conn.keys('*')

[b'carats', b'pie', b'fever', b'song', b'zoo', b'secret', b'cordial']

In [132]:
conn.lrange('zoo', 0, -1)

[b'duck',
 b'alligator',
 b'marmoset',
 b'bear',
 b'cassowary',
 b'alligator',
 b'marmoset',
 b'bear',
 b'cassowary']

In [133]:
# 末尾に追加する
conn.rpush('zoo', 'yak')

10

In [134]:
conn.lrange('zoo', 0, -1)

[b'duck',
 b'alligator',
 b'marmoset',
 b'bear',
 b'cassowary',
 b'alligator',
 b'marmoset',
 b'bear',
 b'cassowary',
 b'yak']

In [135]:
# 指定したオフセットの値を得る
conn.lindex('zoo', 3)

b'bear'

In [136]:
conn.lrange('zoo', 0, 2)

[b'duck', b'alligator', b'marmoset']

In [137]:
# リストを刈り込む。指定されたオフセットの範囲の要素だけが残る
conn.ltrim('zoo', 1, 4)

True

In [138]:
conn.lrange('zoo', 0, -1)

[b'alligator', b'marmoset', b'bear', b'cassowary']

In [139]:
# ハッシュ
# 辞書と似ていて、文字列しか格納できない。深さ１レベル
# songというRedisハッシュをつくって操作する
# do, reフィールドを同時に設定する
conn.hmset('song', {'do': 'a deer', 're': 'about a deer'})

True

In [140]:
# ハッシュ内に1つのフィールドを設定
conn.hset('song', 'mi', 'a note to follow re')

0

In [141]:
# 1つのフィールドを取得する
conn.hget('song', 'mi')

b'a note to follow re'

In [142]:
# 複数のフィールドを取得する
conn.hmget('song', 're', 'do')

[b'about a deer', b'a deer']

In [143]:
# ハッシュのすべてのフィールドのキーを取得する
conn.hkeys('song')

[b'do', b're', b'mi', b'fa']

In [144]:
conn.hmget('song', conn.hkeys('song'))

[b'a deer',
 b'about a deer',
 b'a note to follow re',
 b'a note that rhymes with la']

In [145]:
# ハッシュのすべてのフィールド値を取得する
conn.hvals('song')

[b'a deer',
 b'about a deer',
 b'a note to follow re',
 b'a note that rhymes with la']

In [146]:
# ハッシュフィールドの数を取得する
conn.hlen('song')

4

In [147]:
# ハッシュ内のすべてのフィールドのキーと値を取得する
conn.hgetall('song')

{b'do': b'a deer',
 b'fa': b'a note that rhymes with la',
 b'mi': b'a note to follow re',
 b're': b'about a deer'}

In [148]:
# キーがまだなければ、フィールドを設定する
conn.hset('song', 'fa', 'a note that rhymes with la')

0

In [149]:
conn.hgetall('song')

{b'do': b'a deer',
 b'fa': b'a note that rhymes with la',
 b'mi': b'a note to follow re',
 b're': b'about a deer'}

In [151]:
# 集合
# Pythonの集合とよく似ている
# 一個または複数の追加
conn.sadd('zoo1', 'duck', 'goat', 'turkey')

3

In [152]:
# 値の数の取得
conn.scard('zoo1')

3

In [153]:
conn.keys('*')

[b'carats', b'pie', b'fever', b'song', b'zoo', b'zoo1', b'secret', b'cordial']

In [154]:
# すべての値を取得する
conn.smembers('zoo1')

{b'duck', b'goat', b'turkey'}

In [155]:
# 集合から取り除く
conn.srem('zoo1', 'turkey')

1

In [156]:
conn.smembers('zoo1')

{b'duck', b'goat'}

In [157]:
# 集合演算を実行するために、もうひとつ集合を作る
conn.sadd('better_zoo1', 'tiger', 'wolf', 'duck')

3

In [158]:
# zoo1とbetter_zoo1の積集合を取得する
conn.sinter('zoo1', 'better_zoo1')

{b'duck'}

In [159]:
# zoo1とbetter_zoo1の積集合を取得して集合fowl_zoo1に結果を格納する
conn.sinterstore('fowl_zoo1', 'zoo1', 'better_zoo1')

1

In [160]:
conn.smembers('fowl_zoo1')

{b'duck'}

In [161]:
# zoo1とbetter_zoo1の和集合を取得する
conn.sunion('zoo1', 'better_zoo1')

{b'duck', b'goat', b'tiger', b'wolf'}

In [162]:
# 和集合演算を実行し、結果をfabulous_zoo1に格納する
conn.sunionstore('fabulous_zoo1', 'zoo1', 'better_zoo1')

4

In [163]:
conn.smembers('fabulous_zoo1')

{b'duck', b'goat', b'tiger', b'wolf'}

In [164]:
# zoo1にあってbetter_zoo1にないもの。差集合を取得し、sdiffstore()を使って差集合をzoo1_sale集合に格納する
conn.sdiff('zoo1', 'better_zoo1')

{b'goat'}

In [165]:
conn.sdiffstore('zoo1_sale', 'zoo1', 'better_zoo1')

1

In [166]:
conn.smembers('zoo1_sale')

{b'goat'}

In [167]:
# ソート済み集合（zset）
# 一意の集合、唯一の不動小数点を持った集合
# 用途 -> スコアボード、副インデックス、タイムライン（タイムスタンプとスコアとして使う）

In [168]:
# タイムスタンプを使って、ユーザーのログインを監視する
import time
now = time.time()
now # UNIX時間

1516543695.977958

In [169]:
# ゲストの追加
conn.zadd('logins', 'smeagol', now)

1

In [170]:
# 5分後に別のゲストがやってくる
conn.zadd('logins', 'sauron', now+(5*60))

1

In [171]:
# 2時間後に次のゲスト
conn.zadd('logins', 'bilbo', now+(2*60*60))

1

In [172]:
conn.zadd('logins', 'treebeard', now+(24*60*60))

1

In [173]:
# bilboがやってきたのは何番か
conn.zrank('logins', 'bilbo')

2

In [174]:
# それはいつか
conn.zscore('logins', 'biblo')

In [175]:
# 全員をログイン順に見てみる
conn.zrange('logins', 0, -1)

[b'smeagol', b'sauron', b'bilbo', b'treebeard']

In [176]:
# 時間付きで
conn.zrange('logins', 0, -1, withscores=True)

[(b'smeagol', 1516543695.977958),
 (b'sauron', 1516543995.977958),
 (b'bilbo', 1516550895.977958),
 (b'treebeard', 1516630095.977958)]

In [177]:
# ビット　大量の数値を少ないスペースで高速に処理できる
# Redisの集合を使うという方法もあるが、ユーザーに数値によるIDを割り当てているなら、ビットを使った方がコンパクトで高速

In [178]:
# 個々の日に対してビットセットを作る。このテストでは3日だけを使う
days = ['2013-0225', '2013-02-26', '2013-02-27']
big_spender = 1089
tire_kicker = 40459
late_joiner = 550212

In [186]:
# 初日2013-02-25にbig_spenderとtire_kickerがログイン
conn.setbit(days[0], big_spender, 1)
conn.setbit(days[0], tire_kicker, 1)

0

In [187]:
# 翌日2013-02-26にもbig_spenderがログイン
conn.setbit(days[1], big_spender, 1)

1

In [188]:
# 最終日2013-02-27にbig_spenderと信じのlate_joinerがログイン
conn.setbit(days[2], big_spender, 1)
conn.setbit(days[2], late_joiner, 1)

1

In [189]:
# ３日間の各日の訪問者数を確認
for day in days:
    print(conn.bitcount(day))

2
1
2


In [190]:
# 特定の日に特定のユーザーが来ているか確認
conn.getbit(days[1], tire_kicker)

0

In [191]:
# 毎日ログインしていたユーザーは何人イルカ
conn.bitop('and', 'everyday', *days)

68777

In [192]:
conn.bitcount('everyday')

1

In [193]:
# それがだれだったのか
conn.getbit('everyday', big_spender)

1

In [194]:
# 3日間のユーザーの合計
conn.bitop('or', 'alldays', *days)

68777

In [195]:
conn.bitcount('alldays')

3

In [196]:
# キャッシュと有効期限
# Redisのすべてのキーには寿命、有効期限がある。デフォルトでは永遠だが、expire()関数で期限を指定できる
# 有効期限は秒単位で指定できる
import time
key = 'now you see it'
conn.set(key, 'but not for long')

True

In [197]:
conn.expire(key, 5)

True

In [198]:
conn.ttl(key)

In [199]:
conn.get(key)

In [200]:
time.sleep(6)

In [201]:
conn.get(key)
# expireat()関数をつがえばUNIX時間にキーが無効になる 使える

In [202]:
conn.keys('*')

[b'fabulous_zoo1',
 b'logins',
 b'2013-02-27',
 b'zoo1_sale',
 b'fever',
 b'better_zoo1',
 b'zoo1',
 b'2013-02-26',
 b'alldays',
 b'2013-0225',
 b'secret',
 b'cordial',
 b'everyday',
 b'carats',
 b'pie',
 b'song',
 b'zoo',
 b'fowl_zoo1']

In [209]:
# すべてのキーを削除
for key in conn.keys('*'):
    conn.delete(key)

In [210]:
conn.keys('*')

[]